<a href="https://colab.research.google.com/github/andradhf/Legisight/blob/petitum/Document_Clasification_SahabatAIModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##import library


In [1]:
!pip install PyPDF2==3.0.1 transformers accelerate torch huggingface_hub
import re
import pandas as pd
import requests
import torch
from PyPDF2 import PdfReader
from google.colab import files
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import BytesIO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

#passing to llm


In [2]:
notebook_login()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
model_id = "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

##upload files

In [4]:
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]
print("Dokumen :", csv_path)
df = pd.read_csv(csv_path)

Saving splp_perkara_202504150803.csv to splp_perkara_202504150803.csv
Dokumen : splp_perkara_202504150803.csv


In [5]:
df = df.sample(n=10, random_state=42)

##Function process

In [6]:
komisi_list = """
Berikut adalah daftar 13 Komisi DPR RI:
1. Komisi I: Pertahanan, luar negeri, intelijen, komunikasi dan informatika.
2. Komisi II: Pemerintahan dalam negeri, aparatur negara, dan reformasi birokrasi.
3. Komisi III: Hukum, HAM, dan keamanan.
4. Komisi IV: Pertanian, lingkungan hidup, kehutanan dan kelautan.
5. Komisi V: Infrastruktur dan transportasi.
6. Komisi VI: Perdagangan, koperasi, UMKM, dan BUMN.
7. Komisi VII: Energi, riset dan teknologi, dan lingkungan hidup.
8. Komisi VIII: Sosial, agama, dan pemberdayaan perempuan.
9. Komisi IX: Kesehatan, ketenagakerjaan, dan transmigrasi.
10. Komisi X: Pendidikan, olahraga, pariwisata, dan kebudayaan.
11. Komisi XI: Keuangan dan perbankan.
12. Komisi XII: Ekonomi kreatif dan digital.
13. Komisi XIII: Komunikasi, penyiaran dan literasi digital.
"""

In [14]:
def extract_text_from_pdf_bytes(pdf_path):
  reader = PdfReader(BytesIO(pdf_path))
  full_text = ""
  for page in reader.pages:
    full_text += page.extract_text()+"\n"
  return full_text

def map_status_to_group(status):
    status = status.lower()
    if any(s in status for s in ["mengabulkan seluruhnya", "mengabulkan sebagian", "dikabulkan sebagian",""]):
        return "Putusan Dikabulkan"
    elif any(s in status for s in ["menolak seluruhnya", "ditolak"]):
        return "Putusan Ditolak"
    elif "tidak dapat diterima" in status:
        return "Putusan Tidak Dapat Diterima"
    elif any(s in status for s in ["gugur ", "ditarik kembali"]):
        return "Putusan Gugur"
    elif any(s in status for s in ["mengabulkan sebagian (ditolak)", "mengabulkan sebagian (tidak dapat diterima)","mengabulkan sebagian (ditolak dan tidak dapat diterima)", "mengabulkan sebagian (ditolak, tidak dapat diterima dan tidak berwenang)","mengabulkan sebagian (ditolak, tidak dapat diterima, tidak berwenang dan gugur)", "menolak sebagian (tidak dapat diterima)", "menolak sebagian (tidak dapat diterima dan gugur)", "menolak sebagian (tidak dapat diterima dan tidak berwenang)" ]):
        return "Putusan Campuran"
    elif "tidak berwenang" in status:
        return "Putusan Tidak Berwenang"
    elif any(s in status for s in ["sela", "sela / provisi"]):
        return "Putusan Sela / Provisi"
    else:
        return "Status Tidak Valid / Belum Diisi"

def extract_petitum_section(text):
  text_cleaned = re.sub(r'\s+', ' ', text, flags=re.MULTILINE).lower()

  match = re.search(r'(petitum.*?)pertimbangan hukum', text_cleaned, re.DOTALL)
  if match:
    petitum_text = match.group(1).strip()
    return petitum_text
  return None

def extract_amar_putusan_section(text):
    text_cleaned = re.sub(r'\s+', ' ', text, flags=re.MULTILINE).lower()
    match = re.search(r'5\.\s*Amar Putusan(.*?)(?=6\.\s*pendapat berbeda|ttd)', text_cleaned, re.DOTALL | re.IGNORECASE)

    if match:
      amar_text = match.group(1).strip()
      return amar_text
    return None

def generate_prompt_petitum(petitum_text, komisi_list):
    return f"""
Anda adalah seorang ahli hukum yang berpengalaman dalam menganalisis dan mengklasifikasikan dokumen hukum, khususnya putusan pengadilan. Anda memiliki pemahaman mendalam mengenai tugas dan ruang lingkup dari 13 komisi di DPR RI, serta kemampuan untuk menilai relevansi konteks permasalahan dalam teks hukum.

Tugas Anda adalah mengklasifikasikan sebuah teks putusan pengadilan ke dalam salah satu dari 13 komisi DPR RI.

Teks putusan pengadilan:
{petitum_text}

{komisi_list}

Saat mengklasifikasikan, harap pertimbangkan konteks permasalahan yang terdapat dalam teks dan bagaimana hal itu terkait dengan tugas dan ruang lingkup masing-masing komisi. Pastikan untuk memberikan alasan singkat mengenai pilihan Anda.
"""

def generate_prompt_amar(amar_text, komisi_list):
    return f"""
Anda adalah seorang ahli hukum yang berpengalaman dalam menganalisis dan mengklasifikasikan dokumen hukum, khususnya putusan pengadilan. Anda memiliki pemahaman mendalam mengenai tugas dan ruang lingkup dari 13 komisi di DPR RI, serta kemampuan untuk menilai relevansi konteks permasalahan dalam teks hukum.

Tugas Anda adalah mengklasifikasikan sebuah teks putusan pengadilan ke dalam salah satu dari 13 komisi DPR RI.

Teks putusan pengadilan:
{amar_text}

{komisi_list}

Saat mengklasifikasikan, harap pertimbangkan konteks permasalahan yang terdapat dalam teks dan bagaimana hal itu terkait dengan tugas dan ruang lingkup masing-masing komisi. Pastikan untuk memberikan alasan singkat mengenai pilihan Anda.
"""


def classify_text_with_llm(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    input_length = inputs['input_ids'].shape[1]
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.2,
            do_sample=False,
        )
    generated_tokens = outputs[0][input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    match = re.search(r'\bkomisi\s+(?:[ivx]+)', generated_text, re.IGNORECASE)
    return match.group(0).strip() if match else generated_text.strip()


In [15]:
komisi_list_result = []

for i, row in df.iterrows():
    try:
        print(f"Memproses dokumen ke-{i+1}")
        status = row["status"]
        status_group = map_status_to_group(status)
        url = row["supporting_file"]
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception("Gagal download PDF.")
        pdf_text = extract_text_from_pdf_bytes(response.content)

        if status_group in ["Putusan Dikabulkan", "Putusan Campuran", "Putusan Sela / Provisi"]:
            amar_text = extract_amar_putusan_section(pdf_text)
            if not amar_text:
                komisi_list_result.append("Amar tidak ditemukan")
                continue
            prompt = generate_prompt_amar(amar_text, komisi_list)
            hasil_klasifikasi = classify_text_with_llm(prompt)

        elif status_group in ["Putusan Ditolak", "Putusan Tidak Dapat Diterima"]:
            petitum_text = extract_petitum_section(pdf_text)
            if not petitum_text:
                komisi_list_result.append("Permohonan gugur")
                continue
            prompt = generate_prompt_petitum(petitum_text, komisi_list)
            hasil_klasifikasi = classify_text_with_llm(prompt)

        elif status_group in ["Putusan Gugur", "Putusan Tidak Berwenang", "Status Tidak Valid / Belum Diisi"]:
            hasil_klasifikasi = status  # return status apa adanya

        else:
            hasil_klasifikasi = f"Status tidak dikenali: {status}"

        komisi_list_result.append(hasil_klasifikasi)

    except Exception as e:
        komisi_list_result.append(f"Error: {str(e)}")

df["komisi"] = komisi_list_result
df.to_csv("hasil_klasifikasi_petitum.csv", index=False)
print("✅ Proses selesai. File disimpan: hasil_klasifikasi_petitum.csv")


Memproses dokumen ke-394


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-3563


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-552


...

Memproses dokumen ke-1566


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-2085


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Proses selesai. File disimpan: hasil_klasifikasi_petitum.csv


In [16]:
hasil_df = pd.read_csv("/content/hasil_klasifikasi_petitum.csv")
hasil_df

,id,putusancode,no_perkara,pokok_perkara,pemohon,amar_putusan,download_count,supporting_file,date,date_dpr,keywords,status,link_hit,status_dpr,user_input,tanggal_input,user_update,tanggal_update,komisi
0,394,4168,147/PUU-XXI/2023,Pengujian Materiil (Pengajuan Kembali) Undang-...,"Dr. H. Marion, S.H., M.H.",Menyatakan permohonan Pemohon tidak dapat dite...,1745,https://s.mkri.id/public/content/persidangan/p...,2024-01-16T15:07:00+07:00,2024-01-16 15:07:00,"usia minimal calon wakil presiden, empat puluh...",Tidak Dapat Diterima,https://api-test.mkri.id/putusan/download/4168,1,ibnu.hisyam,2024-08-07 12:36:10,ibnu.hisyam,2024-08-07 12:36:10,Komisi III
1,3563,4783,247/PHPU.GUB-XXIII/2025,Perselisihan Hasil Pemilihan Umum Gubernur Sum...,Edy Rahmayadi dan Hasan Basri Sagala,Dalam Eksepsi:<br />1.\tMengabulkan eksepsi Te...,132,https://s.mkri.id/public/content/persidangan/p...,2025-02-04T12:53:00+07:00,2025-02-04 12:53:00,Perselisihan Hasil Pemilihan Umum Gubernur Sum...,Tidak Dapat Diterima,https://api-test.mkri.id/putusan/download/4783,1,muhammad.wibowo,2025-02-05 09:29:01,muhammad.wibowo,2025-02-05 09:29:01,Komisi III
2,552,3962,95/PUU-XX/2022,Pengujian Undang-Undang Nomor 10 Tahun 2016 te...,H. Muhammad Jafar Sukhairi Nasution,Ditolak,232879,https://s.mkri.id/public/content/persidangan/p...,2022-11-23T14:56:00+07:00,2022-11-23 14:56:00,"masa jabatan kepala daerah, Pilkada serentak t...",Menolak Seluruhnya,https://api-test.mkri.id/putusan/download/3962,1,ibnu.hisyam,2024-08-07 12:36:12,ibnu.hisyam,2024-08-07 12:36:12,Komisi III
3,1296,3120,237-07-28/PHPU.DPR-DPRD/XVII/2019,Perselisihan Hasil Pemilihan Umum DPR-DPRD Pro...,Partai Berkarya,Menyatakan Permohonan Pemohon gugur.,230310,https://s.mkri.id/public/content/persidangan/p...,2019-08-06T10:39:00+07:00,2019-08-06 10:39:00,Perselisihan Hasil Pemilihan Umum DPR-DPRD Pro...,Gugur,https://api-test.mkri.id/putusan/download/3120,1,ibnu.hisyam,2024-08-07 12:36:23,ibnu.hisyam,2024-08-07 12:36:23,Amar tidak ditemukan
4,819,3625,71/PHP.BUP-XIX/2021,Perselisihan Hasil Pemilihan Bupati Kabupaten ...,9999,<div>Dalam Eksepsi:</div><div>1. Menyatakan ek...,230811,https://s.mkri.id/public/content/persidangan/p...,2021-02-17T15:20:00+07:00,2021-02-17 15:20:00,Perselisihan Hasil Pemilihan Bupati Kabupaten ...,Tidak Dapat Diterima,https://api-test.mkri.id/putusan/download/3625,1,ibnu.hisyam,2024-08-07 12:36:17,ibnu.hisyam,2024-08-07 12:36:17,Komisi III
5,2663,1007,40/PUU-IX/2011,Pengujian Undang-Undang Nomor 6 Tahun 2011 ten...,1. Rico Pandejrot; 2. Afrian Bondjol; 3. Yuliu...,Dikabulkan,231033,https://s.mkri.id/public/content/persidangan/p...,2012-02-08T16:00:00+07:00,2012-02-08 16:00:00,penyelidikan; dugaan tindak pidana; hak asasi ...,Mengabulkan Seluruhnya,https://api-test.mkri.id/putusan/download/1007,1,ibnu.hisyam,2024-08-07 12:36:45,ibnu.hisyam,2024-08-07 12:36:45,Komisi III
6,568,3918,84/PUU-XX/2022,Pengujian Undang-Undang Nomor 7 Tahun 2014 ten...,"Rizky Puguh Wibowo, Zainal Hudha Purnama, dan ...",Ditolak,232423,https://s.mkri.id/public/content/persidangan/p...,2022-09-29T16:40:00+07:00,2022-09-29 16:40:00,"barang, e-book, robot trading, skema piramida\r\n",Menolak Sebagian (Tidak Dapat Diterima),https://api-test.mkri.id/putusan/download/3918,1,ibnu.hisyam,2024-08-07 12:36:12,ibnu.hisyam,2024-08-07 12:36:12,Komisi VI
7,1728,2618,45/PHP.GUB-XV/2017,Perselisihan Hasil Pemilihan Gubernur dan Waki...,"1.H. Rano Karno, S.Ip. 2.H. Embay Mulya Syarie...",1.Mengabulkan eksepsi Termohon dan eksepsi Pih...,230583,https://s.mkri.id/public/content/persidangan/p...,2017-04-04T16:02:00+07:00,2017-04-04 16:02:00,Perselisihan Hasil Pemilihan Gubernur dan Waki...,Tidak Dapat Diterima,https://api-test.mkri.id/putusan/download/2618,1,ibnu.hisyam,2024-08-07 12:36:31,ibnu.hisyam,2024-08-07 12:36:31,Komisi III
8,1566,2781,77/PUU-XV/2017,Pengujian Undang-Undang Nomor 30 Tahun 2014 te...,Richard Christoforus Massa \r\n\r\nKuasa Pemoh...,Ditolak,230779,https://s.mkri.id/public/content/persidangan/p...,2018-05-0